# Planteamiento del problema

#### Una de las problematicas del sector paciente sanitario es la ocultación de información de prácticas no saludables. El objetivo del modelo final es determinar mediante características físicas de pacientes si es fumador-bebedor. Para ello se toma un dataset completo de características físicas de pacientes con la confirmación de si son o no bebedores y la clasificación de fumadores (1 nunca fumó, 2 fumó pero dejó, 3 aún fuma). 
#### Estamos ante un problema de clasificación. El modelo final deberá predecir 0-1 si bebe o no y otro 1-2-3 dependiendo del estado de fumar.
#### El archivo train_data contiene alrededor de un 70% de los pacientes para entrenar el modelo, el test_data el resto de pacientes.
#### El código está compuesto por, una parte inicial de tratamiento de datos (Es un data set bien curado y fácil de manejar). A continuación representación de gráficos y finalmente una búsqueda de modelos de ML. Primero se utiliza un modo más manual con bucles for y finalmente un proceso automatizado mediante GridSearch.

# Librerias

#### Inicialmente se importan las librerias para trabajar con el dataset, representar gráficos y ML.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Tratamiento de datos

### Carga de datos de entrenamiento y test.

#### Se lee la información que contienen los archivos csv que anteriormente ha sido separada, train_data para entrenar el modelo y test_data para comprobar que tan fiable es el modelo.

In [ ]:
train_path = os.path.join('train_data.csv')
test_path = os.path.join('test_data.csv')
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

#### Mediante .shape se comprueba la cantidad de pacientes y características de las que se compone cada DF para comprobar que la separación de datos fue correcta.

In [ ]:
print(train_data.shape)
print(test_data.shape)

#### En primer lugar mediante .info se mira el tipo de información que contienen las columnas. Se puede apreciar que es una buena base de datos ya que no tiene datos faltantes. Hay datos tipo entero, flotante y object, los cuales más adelante habrá que ver cómo procesarlos.
#### Contiene información de una revisión básica como edad, sexo, peso, altura, presión... Y de análisis como creatinina, colesterol...  

##### En el caso que hubieran existido datos faltantes Nan, se tendrían que filtrar. En el caso de ser datos de columnas sin relevancia para el modelo se eliminarían directamente. En caso contrario si hubieran sido pocos, como hay muchos datos, lo más probable es que se hubieran eliminado también. En el caso de haber sido un porcentaje más elevado, dependiendo de los valores faltantes y del resto, podría haberse utilizado la media del resto de datos por ejemplo.
##### La mayoría de datos ya son numéricos, pero en el caso que hubieran sido más característicos, como más abajo, se categorizarían por números o si no fueran relevantes se eliminarían.

In [ ]:
train_data.info()

In [ ]:
test_data.info()

#### Con .describe se quierr ver en qué rangos está la información en general, como por ejemplo se han tomado muestras de gente de entre 20 y 85 años y la media sale por debajo de los 50, lo cuál indica que hay más datos de gente joven que no de la 3a edad. Hay más datos de hombres que de mujeres pero no una gran diferencia, alrededor del 53%.

In [ ]:
train_data.describe(include='all')

In [ ]:
test_data.describe(include='all')

#### En las tablas anteriores se ve que hay valores irreales como un máximo de cintura de 999, actualmente el record guiness es de unos 300, el valor máximo registrado de triglicéridos es de unos 4000 mg/dL, de colesterol 3000 y otros valores que distan mucho de la mayoria del resto de valores. Por lo que esos valores se tomarán como si fueran nulos eliminando dichas filas tanto de train_data como de test_data.

In [ ]:
train_data.drop(train_data[(train_data['waistline'] >300)].index, inplace=True)
train_data.drop(train_data[(train_data['triglyceride'] >4500)].index, inplace=True)
train_data.drop(train_data[(train_data['LDL_chole'] >3000)].index, inplace=True)
train_data.drop(train_data[(train_data['SGOT_AST'] >3000)].index, inplace=True)
test_data.drop(test_data[(test_data['waistline'] >300)].index, inplace=True) 
test_data.drop(test_data[(test_data['triglyceride'] >4500)].index, inplace=True)
test_data.drop(test_data[(test_data['LDL_chole'] >3000)].index, inplace=True)
test_data.drop(test_data[(test_data['SGOT_AST'] >3000)].index, inplace=True)

#### Aquí ya podemos apreciar que ha eliminado ciertas filas y los máximos que era probable que no fueran reales han sido eliminados.

In [ ]:
train_data.describe(include='all')

## Transformación de datos object en enteros

#### Para poder graficar y posteriormente trabajar machinelearning, se han de transformar los datos caracaterísticos en numéricos como el sexo y si fuman o no.

#### Se realiza una copia de df para no tener que repetir la limpieza anterior si hubiera que realizar cualquier cambio.

In [ ]:
df_train = train_data.copy()
df_test = test_data.copy()

#### A continuación hay una función para codificar los datos característicos.

In [ ]:
def encode_features(df_train, df_test, features):
    '''
    Union de los DataFrames para que coincidan los cambios en todos los DF por separado
    
    Input: 2 pandas DataFrames y una lista de caracteristicas
    Output: 2 DataFrames con los datos característicos transformados en numeros  
    '''
    df_combined = pd.concat([df_train, df_test])
    for feature in features:
        le=preprocessing.LabelEncoder()
        le.fit(df_combined[feature])
        if feature in df_train.columns:
            df_train[feature]=le.transform(df_train[feature])
        else:
            pass
        if feature in df_test.columns:
            df_test[feature]=le.transform(df_test[feature])
        else:
            pass
    return

#### Por el momento se aplica la función a la columna de bebedor/no bebedor para poder graficar. Más adelante cuando vayamos a la parte de machinelearning se volverá a aplicar la función para el resto de columnas.

In [ ]:
features=['DRK_YN']
encode_features(df_train,df_test,features)

## Gráficos

#### Una vez transformados los datos en números se pueden realizar gráficos para analizar. La función aplica un 1 al bebedor y un 0 a no bebedor.
#### A continuación aparecen varios gráficos que ayudan a ver la influencia de varias características.

#### El gráfico más básico es ver la relación con edad y sexo. Como son muchos datos al pc le cuesta mucho calcular segun que gráficos y otros aunque los calcule relativamente más fácil, no se distingue bien la información y realizamos un par de gráficos para comparar y ver que la tendencia más o menos se asemeja del DF completo al 1000 intermedios.

In [ ]:
grf=sns.pointplot(x='age', y='DRK_YN', hue='sex', data=df_train)
grf.set_xlabel("Edad", fontsize = 10)
grf.set_ylabel("Bebedor", fontsize = 10)

In [ ]:
grf=sns.pointplot(x='age', y='DRK_YN', hue='sex', data=df_train[40000:50000])
grf.set_xlabel("Edad", fontsize = 10)
grf.set_ylabel("Bebedor", fontsize = 10)

#### Este tipo de gráfico con muchos datos ya se emborrona y empieza a perder claridad por lo que se prueban zonas pequeñas del DF viendo que se asemejan bastante.
#### Se ve que hay más fumadores jóvenes y una relación bastante directa con el aumento de cintura.

In [ ]:
sns.jointplot(x='age', y='waistline', hue='SMK_stat_type_cd',  data=df_train[500:800])
plt.show()

In [ ]:
sns.jointplot(x='age', y='waistline', hue='SMK_stat_type_cd',  data=df_train[20000:20300])
plt.show()

In [ ]:
sns.jointplot(x='age', y='waistline', hue='SMK_stat_type_cd',  data=df_train[200000:200300])
plt.show()

In [ ]:
grf=sns.pointplot(x='age', y='SMK_stat_type_cd', hue='sex', data=df_train)
grf.set_xlabel("Edad", fontsize = 10)
grf.set_ylabel("Fumador", fontsize = 10)

#### Hay datos que no parecen influyentes en el caso, como la vision. Vamos a graficar a ver si se ve algo, pero se intuye que producirá ruido.

In [ ]:
sns.jointplot(x='age', y='sight_left', hue='SMK_stat_type_cd',  data=df_train[200000:200300])
plt.show()

In [ ]:
sns.jointplot(x='age', y='sight_left', hue='DRK_YN',  data=df_train[200000:200300])
plt.show()

In [ ]:
sns.jointplot(x='age', y='hear_right', hue='SMK_stat_type_cd',  data=df_train[200000:200300])
plt.show()

In [ ]:
sns.jointplot(x='age', y='hear_right', hue='DRK_YN',  data=df_train[100000:100300])
plt.show()

In [ ]:
grf=sns.pointplot(x='age', y='hear_right', hue='DRK_YN', data=df_train)
grf.set_xlabel("Oido", fontsize = 10)
grf.set_ylabel("Bebedor", fontsize = 10)

#### Los gráficos de arriba, son una muestra de varias franjas del DF, de ellas se deduce que no hay una relación evidente de la falta de visión con fumador-bebedor. Lo mismo para el oído, se ve en el último gráfico que aumenta la falta de audición al aumentar la edad, pero no hay diferencia entre la línea de bebedores.
#### Por ello se decide eliminar las columnas de audición y visión, porque pueden inducir a error más que ayudar.

In [ ]:
df_train=df_train.drop(['sight_left','sight_right','hear_left','hear_right'], axis=1)
df_test=df_test.drop(['sight_left','sight_right','hear_left','hear_right'], axis=1)

#### Por último se pasan el resto de datos característicos con la función anterior a números para poder empezar con ML.

In [ ]:
features1=['sex']
encode_features(df_train,df_test,features1)

In [ ]:
df_train

# MachineLearning

#### Las columnas SMK y DRK seeliminan porque son las variables objetivo en x para que estudie el resto de columnas, y las 2 columnas de variables objetivo se ponen por separado para hacer 2 modelos distintos.
#### De los datos de entrenamiento se utiliza un 85% para entrenar y un 15% para validar, y como no se sabe qué cantidad hay en los datos de cada uno de la clasificación 1,2,3 de fumadores, se utiliza el "stratify = y" para asegurar que se mantenga la proporción tanto en entrenamiento como en test.

In [ ]:
x = df_train.drop(['SMK_stat_type_cd','DRK_YN'], axis=1)
y = df_train['SMK_stat_type_cd']
z = df_train['DRK_YN']

validation_split = 0.15
seed = 42
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=validation_split, random_state=seed, stratify=y)

#### Modelo de arbol de clasificación y entrenamos el modelo para la clasificación de fumador(1,2,3)

In [ ]:
model_smk = DecisionTreeClassifier()

model_smk.fit(x_train,y_train)

#### Modelo de arbol de clasificación y entrenamos el modelo para la clasificación de bebedor(0,1).

In [ ]:
x_train, x_validation, z_train, z_validation = train_test_split(x, z, test_size=validation_split, random_state=seed, stratify=z)

In [ ]:
model_drk = DecisionTreeClassifier()

model_drk.fit(x_train,z_train)

In [ ]:
pred_smk = model_smk.predict(x_validation)

In [ ]:
pred_drk = model_drk.predict(x_validation)

#### Comparación de datos de la predicción con los reales.

In [ ]:
accuracy_score(pred_smk, y_validation)

In [ ]:
accuracy_score(pred_drk, z_validation)

#### Se utiliza la matriz de confusión para ver la relación de errores, comparando los Ture positive, True negative, False positive y False negative. Es decir los valores que el modelo determina que son positivos y realmente son positivos, igualmente para los negativos y por el lado contrario, los que determina positivos pero son negativos y viceversa.

In [ ]:
print(confusion_matrix(y_validation,pred_smk))

In [ ]:
print(confusion_matrix(z_validation,pred_drk))

In [ ]:
print(classification_report(y_validation,pred_smk))

In [ ]:
print(classification_report(z_validation,pred_drk))

#### Calculamos el accuracy con los datos de test para ver si el modelo generaliza bien.

In [ ]:
y_test = df_test['SMK_stat_type_cd']
z_test = df_test['DRK_YN']
x_test = df_test.drop(['SMK_stat_type_cd','DRK_YN'], axis=1)

pred_smk_test = model_smk.predict(x_test)
pred_drk_test = model_drk.predict(x_test)

In [ ]:
accuracy_score(pred_smk_test,y_test)

In [ ]:
accuracy_score(pred_drk_test,z_test)

#### A continuación se programa un modelo KNeighbors con un bucle para buscar qué semilla iría mejor, y ver si se puede mejorar el algoritmo.

In [ ]:
k_range = list(range(1,25))
scores_smk=[]
for k in k_range:
    knn_smk = KNeighborsClassifier(n_neighbors=k)
    knn_smk.fit(x_train,y_train)
    y_pred = knn_smk.predict(x_validation)
    scores_smk.append(accuracy_score(y_validation, y_pred))

print(scores_smk)

#### Un gráfico con la lista de accuracys y las semillas muestra con más claridad cuál es mejor viendo que se mueve entre un rango de 40 y 60 %. Al parecer a partir de una semilla de 18-20 se estabiliza en un 60 %. Se puede ver reflejada la influencia de la semilla y que el modelo KNeighbors da mejor resultado con una semilla superior a 20 al compararla con el 1er modelo del árbol de decisión.

In [ ]:
fig, ax = plt.subplots()
ax.plot(k_range, scores_smk)
ax.set_xlabel('Valores de semillas')
ax.set_ylabel('Accuracy')
plt.show()

In [ ]:
knn_smk = KNeighborsClassifier(24)
knn_smk.fit(x_train,y_train)
y_pred = knn_smk.predict(x_validation)
accuracy_score(y_validation, y_pred)

#### Mismo porceso para el modelo de bebedores. También es un problema de calsificación y aunque el anterior es de 3 clases y éste solo de 2 también se puede ver que éste modelo es mejor que el anterior y alrededor de 18-20 también se estabiliza.

In [ ]:
k_range = list(range(1,25))
scores_drk = []
for k in k_range:
    knn_drk = KNeighborsClassifier(n_neighbors=k)
    knn_drk.fit(x_train,z_train)
    z_pred = knn_drk.predict(x_validation)
    scores_drk.append(accuracy_score(z_validation, z_pred))

print(scores_drk)

In [ ]:
fig, ax = plt.subplots()
ax.plot(k_range, scores_drk)
ax.set_xlabel('Valores de semillas')
ax.set_ylabel('Accuracy')
plt.show()

In [ ]:
knn_drk = KNeighborsClassifier(24)
knn_drk.fit(x_train,z_train)
z_pred = knn_drk.predict(x_validation)
accuracy_score(z_validation, z_pred)

#### Una vez entrenado el modelo se prueban los datos de test con la semilla que mejores resultados daban, que en ambos casos coincide que es la semilla más alta y es la que se ha dejado entrenada tras ver cada gráfico. Como se puede apreciar, el modelo generaliza bien con datos nuevos.

In [ ]:
pred_knn_smk_test = knn_smk.predict(x_test)
pred_knn_drk_test = knn_drk.predict(x_test)

In [ ]:
accuracy_score(pred_knn_smk_test,y_test)

In [ ]:
accuracy_score(pred_knn_drk_test,z_test)

#### A continuación está programado un Cross-Validation para entrenar con todos los datos sin separar datos en train y validation, utilizando varias combinaciones de datos para train y para validación, tanto para el sector de fumadores como el de bebedores. Utilizo 10 iteraciones para cada valor de neighbor. 
#### Tras los entrenos está programado al igual que antes un gráfico para visualizar la influencia de las diferentes semillas.

In [ ]:
k_range = list(range(1,25))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
    k_scores.append(scores.mean())
    
print(k_scores)

In [ ]:
fig, ax = plt.subplots()
ax.plot(k_range, k_scores)
ax.set_xlabel('Valores de semillas')
ax.set_ylabel('Accuracy')
plt.show()

In [ ]:
k_range = list(range(1,25))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train, z_train, cv=10, scoring='accuracy')
    k_scores.append(scores.mean())
    
print(k_scores)

In [ ]:
fig, ax = plt.subplots()
ax.plot(k_range, k_scores)
ax.set_xlabel('Valores de semillas')
ax.set_ylabel('Accuracy')
plt.show()

#### Para acabar se utiliza un modo más efectivo, GridSearch y más de un parámetro.

#### Definición de los parámetros que se quiere buscar, valores de k entre 1-25 y las opciones de peso

In [ ]:
k_range = list(range(1, 25))
weight_options = ['uniform', 'distance']

#### Param-grid, un diccionario, poniendo el nombre para asignarle a esa característica sobre la que iterar.

In [ ]:
param_grid = dict(n_neighbors=k_range, weights=weight_options)
print(param_grid)

#### Instancia del GridSearch pasándole el parámetro n_jobs=-1 para que lance en paralelo los que pueda, aprovechando diferentes núcleos y así se ejecute más rápido.

In [ ]:
grid_smk = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', n_jobs=-1)
grid_smk.fit(x_train, y_train)

In [ ]:
grid_drk = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', n_jobs=-1)
grid_drk.fit(x_train, z_train)

#### Datos a un DataFrame para visualizar mejor los datos. Obteniendo la media de las 10 iteraciones sobre cada una de las combinaciones y la desviación que se genera.

In [ ]:
pd.DataFrame(grid_smk.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

In [ ]:
pd.DataFrame(grid_drk.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

#### Con Grid se puede ver la mejor combinación para el mejor resultado 

In [ ]:
print(grid_smk.best_score_)
print(grid_smk.best_params_)

In [ ]:
print(grid_drk.best_score_)
print(grid_drk.best_params_)

#### Y finalmente se comprueba el modelo con los datos de test

In [ ]:
pred_grid_smk = grid_smk.predict(x_test)

In [ ]:
pred_grid_drk = grid_drk.predict(x_test)

In [ ]:
accuracy_score(pred_grid_smk,y_test)

In [ ]:
accuracy_score(pred_grid_drk,z_test)